In [8]:
# 2. Test Statistical Models
# a. Prepare the dataset for modeling:
# •	Split into training and testing sets:

import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv('D:/Personal/Kifiya 10 Academy/Week 3/data/data.csv')
from sklearn.model_selection import train_test_split

# Check if 'TotalPremium' column exists
if 'TotalPremium' in data.columns:
	X = data.drop('TotalPremium', axis=1)
	y = data['TotalPremium']
else:
	raise KeyError("Column 'TotalPremium' not found in the dataset")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


C:\Users\Almazt\AppData\Local\Temp\ipykernel_2456\318059357.py:9: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('D:/Personal/Kifiya 10 Academy/Week 3/data/data.csv')


In [9]:
# b. Test Models
# •	Linear Regression:

from sklearn.linear_model import LinearRegression

# Clean and convert 'TransactionMonth' to datetime
X_train['TransactionMonth'] = pd.to_datetime(X_train['TransactionMonth'], errors='coerce')
X_test['TransactionMonth'] = pd.to_datetime(X_test['TransactionMonth'], errors='coerce')

# Drop rows with NaT values in 'TransactionMonth'
X_train = X_train.dropna(subset=['TransactionMonth'])
X_test = X_test.dropna(subset=['TransactionMonth'])

# Ensure y_train and y_test are aligned with X_train and X_test
y_train = y_train[X_train.index]
y_test = y_test[X_test.index]

# Convert all columns to numeric, coerce errors to NaN, and fill NaN with 0
X_train = X_train.apply(pd.to_numeric, errors='coerce').fillna(0)
X_test = X_test.apply(pd.to_numeric, errors='coerce').fillna(0)

# Fit the model
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)


In [10]:
# •	Decision Trees:

from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth=5, random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

In [11]:
# •	Gradient Boosting (XGBoost):

%pip install xgboost
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

In [12]:
# 3. Evaluate Models
# Evaluate using metrics such as MAE, RMSE, and R2R^2R2:

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Evaluate Linear Regression
print("Linear Regression Metrics:")
print("MAE:", mean_absolute_error(y_test, y_pred_lr))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_lr)))
print("R²:", r2_score(y_test, y_pred_lr))

# Repeat for Decision Trees and XGBoost


Linear Regression Metrics:
MAE: 90.669181197405
RMSE: 165.9928525042663
R²: 0.0012524406857028492


In [ ]:
# 4. Model Interpretability
# Use SHAP or LIME to explain predictions and gain actionable insights.
# a. Using SHAP:


import shap
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split

# Initialize SHAP explainer
explainer = shap.Explainer(xgb, X_test)
shap_values = explainer(X_test)

# Summary plot
shap.summary_plot(shap_values, X_test)


TypeError: The passed model is not callable and cannot be analyzed directly with the given masker! Model: <module 'xgboost' from 'c:\\Users\\Almazt\\AppData\\Local\\anaconda3\\envs\\myenv\\lib\\site-packages\\xgboost\\__init__.py'>

In [15]:
# b. Using LIME:

%pip install lime
from lime.lime_tabular import LimeTabularExplainer

# Create a LIME explainer
explainer = LimeTabularExplainer(X_train.values, 
                                 feature_names=X_train.columns, 
                                 class_names=['target'],
                                 mode='regression')

# Explain a single prediction
exp = explainer.explain_instance(X_test.iloc[0].values, xgb.predict, num_features=5)
exp.show_in_notebook()

AttributeError: module 'xgboost' has no attribute 'predict'